In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./data/segmentations.csv")

rle_dict = (
    df.groupby("ImageId")["EncodedPixels"]
      .apply(list)
      .to_dict()
)

im_ids = list(rle_dict.keys())

im_ids[:3]

['00003e153.jpg', '0001124c7.jpg', '000155de5.jpg']

In [4]:
from sklearn.model_selection import train_test_split

In [21]:
# test_im_ids, _ = train_test_split(im_ids, test_size=0.2, random_state=42)

np.random.seed(42)
test_im_ids = np.random.permutation(im_ids)[:100]


print(f"Number of test images: {len(test_im_ids)}")

test_rle_dict = {k: rle_dict[k] for k in test_im_ids}

keys = list(test_rle_dict.keys())[:5]
{k: test_rle_dict[k] for k in keys}

Number of test images: 100


{np.str_('91a58adff.jpg'): [nan],
 np.str_('00189cc36.jpg'): [nan],
 np.str_('940e117fc.jpg'): [nan],
 np.str_('bfcbefae2.jpg'): [nan],
 np.str_('099c2803e.jpg'): [nan]}

In [18]:
from sam_predictor import get_image_iou_mat

In [19]:
from sam_predictor import evaluate_model
from util.general_utils import average_f_score_of_image, fetch_image_from_dict
from numpy.typing import NDArray
from tqdm import tqdm

In [22]:
im_f2s : NDArray[np.float32] = np.array([], dtype=np.float32)

for img, rles in tqdm(fetch_image_from_dict(test_rle_dict), total=len(test_rle_dict)):
    iou_mat : NDArray[np.float32] = get_image_iou_mat((img, rles))
    f2_score : np.float32 = average_f_score_of_image(iou_mat)
    im_f2s = np.append(im_f2s, f2_score)

100%|██████████| 100/100 [22:02<00:00, 13.23s/it]


In [23]:
im_f2s.mean()

np.float64(0.017086813186813186)